# Análisis Histórico de compras 2022-2028

In [0]:
%sql
SELECT COUNT(DISTINCT clave) as claves
  FROM (
    SELECT clave FROM workspace.default.silver_2023_2024
    UNION ALL
    SELECT clave FROM workspace.default.silver_2025_2026
    UNION ALL
    SELECT clave FROM workspace.default.silver_2027_2028
  ) 

In [0]:
%sql
WITH claves_list AS (
  SELECT DISTINCT clave
  FROM (
    SELECT clave FROM workspace.default.silver_2023_2024
    UNION ALL
    SELECT clave FROM workspace.default.silver_2025_2026
    UNION ALL
    SELECT clave FROM workspace.default.silver_2027_2028
  ) 
)
SELECT
  cl.clave,
  CAST(COALESCE(tab2023.totales_max, 0) AS DOUBLE) AS max_2023_2024,
  CAST(COALESCE(tab2025.totales_max, 0) AS DOUBLE) AS max_2025_2026,
  CAST(COALESCE(tab2027.totales_max, 0) AS DOUBLE) AS max_2027_2028
FROM claves_list cl
FULL OUTER JOIN workspace.default.silver_2023_2024 tab2023
  ON cl.clave = tab2023.clave
FULL OUTER JOIN workspace.default.silver_2025_2026 tab2025
  ON cl.clave = tab2025.clave
FULL OUTER JOIN workspace.default.silver_2027_2028 tab2027
  ON cl.clave = tab2027.clave
ORDER BY
  cl.clave;

In [0]:
%sql
SELECT
  -- individual counts
  (SELECT COUNT(clave) FROM workspace.default.silver_2023_2024) AS claves_2023,
  (SELECT COUNT(clave) FROM workspace.default.silver_2025_2026) AS claves_2025,
  (SELECT COUNT(clave) FROM workspace.default.silver_2027_2028) AS claves_2027,
  -- total sum of all three
  (
    (SELECT COUNT(clave) FROM workspace.default.silver_2023_2024)
    + (SELECT COUNT(clave) FROM workspace.default.silver_2025_2026)
    + (SELECT COUNT(clave) FROM workspace.default.silver_2027_2028)
  ) AS totales;

In [0]:
%sql
WITH t2023 AS (
  SELECT
    clave,
    totales_max,
    CAST('2023-01-01' AS DATE) AS year
  FROM workspace.default.silver_2023_2024
),
t2025 AS (
  SELECT
    clave,
    totales_max,
    CAST('2025-01-01' AS DATE) AS year
  FROM workspace.default.silver_2025_2026
),
t2027 AS (
  SELECT
    clave,
    totales_max,
    CAST('2027-01-01' AS DATE) AS year
  FROM workspace.default.silver_2027_2028
)
SELECT *
FROM t2023
UNION ALL
SELECT * FROM t2025
UNION ALL
SELECT * FROM t2027;



In [0]:
%sql
SELECT
  *
FROM 
  workspace.default.silver_2023_2024
LIMIT 
  10;

In [0]:
%sql
SELECT
  *
FROM 
  workspace.default.silver_2025_2026
LIMIT 
  10;

In [0]:
%sql
SELECT
  *
FROM 
  workspace.default.silver_2027_2028
¿LIMIT 
  10;

## Silver table


In [0]:
# -----------------------------
# 1) Cargar tabla bronze
# -----------------------------
df_bronze = spark.table("workspace.default.bronze_2027_2028")


# -----------------------------
# 5) Agregar columnas basado en prefijos
# -----------------------------

# Define the aggregation families
AGG_FAMILIES = ["imss_bienestar", "ccinshae", "salud_spps"]


# For each family, find matching columns and aggregate them
for family in AGG_FAMILIES:
    # Find columns that match the pattern: family_*_min and family_*_max
    min_cols = [col for col in df_bronze.columns if col.startswith(f"{family}_") and col.endswith("_min")]
    max_cols = [col for col in df_bronze.columns if col.startswith(f"{family}_") and col.endswith("_max")]
    
    # Sum these columns to create aggregated columns
    if min_cols:
        # Create sum expression for min columns
        min_sum_expr = sum([F.col(c) for c in min_cols])
        df_aggregated = df_bronze.withColumn(f"{family}_min", min_sum_expr)
    
    if max_cols:
        # Create sum expression for max columns
        max_sum_expr = sum([F.col(c) for c in max_cols])
        df_aggregated = df_aggregated.withColumn(f"{family}_max", max_sum_expr)

# Now select only the columns you need
# Get the base columns
base_cols = ['clave', 'descripcion', 'imss_min', 'imss_max', 'issste_min', 'issste_max', 
             'pemex_min', 'pemex_max']

# Agregar columnas de familias
for family in AGG_FAMILIES:
    base_cols.extend([f"{family}_min", f"{family}_max"])

# Agregar totales
base_cols.extend(['totales_min', 'totales_max'])

# Limpiar descripcion: remover comillas y normalizar espacios
df_clean = df_aggregated.withColumn(
    'descripcion',
    F.regexp_replace(F.col('descripcion'), '"', '')  # Quitar comillas dobles
).withColumn(
    'descripcion',
    F.regexp_replace(F.col('descripcion'), "'", '')  # Quitar comillas simples
).withColumn(
    'descripcion',
    F.trim(F.col('descripcion'))  # Quitar espacios al inicio/final
).withColumn(
    'descripcion',
    F.regexp_replace(F.col('descripcion'), '\\s+', ' ')  # Normalizar múltiples espacios a uno solo
)

# Seleccionar SOLO las columnas que necesitamos
df_silver_clean = df_clean.select(*base_cols)

df_silver_clean.display()

# -----------------------------
# 7) Persist Silver
# -----------------------------
spark.sql("DROP TABLE IF EXISTS workspace.default.silver_licitacion_info")

(df_silver_clean.write
 .mode("overwrite")
 .format("delta")
 .saveAsTable("workspace.default.silver_licitacion_info"))

print("✅ Silver table created cleanly")

In [0]:
%sql
-- Validación de la agregación. 
WITH validacion AS (
  SELECT
    clave,
    totales_min,
    totales_max,

    /* sums with null-safe coalesce */
    COALESCE(imss_min,0) + COALESCE(issste_min,0) + COALESCE(pemex_min,0)
    + COALESCE(imss_bienestar_min,0) + COALESCE(ccinshae_min,0) + COALESCE(salud_spps_min,0)
      AS sum_min,

    COALESCE(imss_max,0) + COALESCE(issste_max,0) + COALESCE(pemex_max,0)
    + COALESCE(imss_bienestar_max,0) + COALESCE(ccinshae_max,0) + COALESCE(salud_spps_max,0)
      AS sum_max
  FROM workspace.default.silver_licitacion_info
),

deltas AS (
  SELECT
    clave,
    totales_min,
    sum_min,
    (sum_min - COALESCE(totales_min,0)) AS delta_min,

    totales_max,
    sum_max,
    (sum_max - COALESCE(totales_max,0)) AS delta_max
  FROM validacion
)

SELECT *
FROM deltas
WHERE delta_min <> 0 OR delta_max <> 0
ORDER BY ABS(delta_min) DESC, ABS(delta_max) DESC;


In [0]:
out_dir = "dbfs:/Volumes/workspace/default/eseotres/silver_licitacion_info_export_csv"

(spark.table("workspace.default.silver_licitacion_info")
 .coalesce(1)  # single CSV file (ok if not huge)
 .write.mode("overwrite")
 .option("header", "true")
 .csv(out_dir))

print("✅ Exported to:", out_dir)

## Golden Tables

In [0]:
# Read silver table
df_silver = spark.table("workspace.default.silver_licitacion_info")

# Step 1: Select only the columns we need
max_columns = [c for c in df_silver.columns if c.endswith('_max') and not c.startswith('totales')]

print(f"📊 Institution columns found: {max_columns}")

# Step 2: Fill nulls with 0 and ensure all columns are BIGINT
df_selected = df_silver.select(['clave', 'descripcion'] + max_columns)

for col_name in max_columns:
    df_selected = df_selected.withColumn(
        col_name,
        F.coalesce(col(col_name).cast('bigint'), F.lit(0))
    )

# Step 3: Build the CORRECT stack expression
# stack(N, 'name1', value1, 'name2', value2, ...) - alternating string/value pairs
stack_expr = f"stack({len(max_columns)}"

for col_name in max_columns:
    institution_name = col_name.replace('_max', '')
    # This is the key fix: wrap column name in backticks for SQL expression
    stack_expr += f", '{institution_name}', `{col_name}`"

stack_expr += ") as (institucion, cantidad)"

print(f"\n🔧 Stack expression: {stack_expr[:200]}...")  # Debug: see first 200 chars

# Step 4: Apply the transformation
df_golden_cross = df_selected.select(
    'clave',
    'descripcion',
    expr(stack_expr)
)

# Step 5: Clean up - remove null/zero quantities (optional)
df_golden_cross = df_golden_cross.filter(
    (col('cantidad').isNotNull()) & 
    (col('cantidad') > 0)
)

# Step 6: Verify the result
print("\n✅ Golden Cross Table Schema:")
df_golden_cross.printSchema()

print("\n📋 Sample data for one product:")
sample_clave = df_golden_cross.select('clave').first()[0]
df_golden_cross.filter(col('clave') == sample_clave).show(truncate=False)

print(f"\n📊 Total rows: {df_golden_cross.count()}")

# Step 7: Save as golden table
spark.sql("DROP TABLE IF EXISTS workspace.default.gold_institution_distribution")

df_golden_cross.write \
    .mode("overwrite") \
    .format("delta") \
    .saveAsTable("workspace.default.gold_institution_distribution")

print("✅ Golden table created: gold_institution_distribution")

In [0]:

# Exportar tabla golden cruzada
out_dir = "dbfs:/Volumes/workspace/default/eseotres"

(spark.table("workspace.default.gold_institution_distribution")
 .coalesce(1)  # single CSV file (ok if not huge)
 .write.mode("overwrite")
 .option("header", "true")
 .csv(out_dir))

print("✅ Exported to:", out_dir)